Requêtage restaurant

Requete API

In [1]:
import requests
from sqlalchemy import create_engine, text
import pandas as pd
from SQL_user import SQL_user


with open('apikeyrm.txt', 'r') as file: #mettre clé api
    API_KEY = file.read().strip()

QUERY = 'cafeteria toulouse gramont'


# URL de l'API Places

url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={QUERY}&key={API_KEY}&sort_by=rating&maxResults=5"
# Envoyer la requête

response = requests.get(url)
# data = response.json()
data2 = response.json()

data2

{'html_attributions': [],
 'next_page_token': 'AVzFdbmtigiYQwsSTemYqNuJh0_Iw9kfX2GMH2hA9DSwPRLft5vjwOUVPXzGRZ9aQgtAlTw_W_Y9l7asxBT_SgZLyq8583S41JaQ5Wkx0u-57-6_UuWeMu0_B73L2i9xJaqBvsryAooju4CsSbjGKJ_7REDwnWiquAgds6DEYeTJM9FvT_6OypXbDlUuQ8KBW3W0xqnvx0n93xGsNIRNQ_ke42nBhFBI5XKH14YSbQ7GIc63WYuS3K2b0wCjS_q-0xkkDukH2h5EO-NDefQCgEfX8KDqEczHPSgVeYxjiEbqat1ufaQ2LX5kp0sgFfiYqwllm7CdRpJHi7_Gyb9Md3mdoBe6g7pcjyrvNWHMyGugICB3wNaBeinjgZdW6VLFyOgmlVYm0FlYoUmicKcqgfM07_ta_l6CsSxY',
 'results': [{'business_status': 'OPERATIONAL',
   'formatted_address': '2 Chem. de Gabardie C.Cial Auchan, 31200 Toulouse, France',
   'geometry': {'location': {'lat': 43.6316772, 'lng': 1.4859998},
    'viewport': {'northeast': {'lat': 43.63290262989273,
      'lng': 1.487653529892722},
     'southwest': {'lat': 43.63020297010728, 'lng': 1.484953870107278}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://m

In [2]:
sql_user= SQL_user()


# Chaîne de connexion
engine = sql_user.engine

In [ ]:


nb_result = 5

# Utilisation de compréhension de dictionnaire pour créer le dictionnaire de la query
dict_adresses = {
    'restaurant_id': [result['name'] + str(result['geometry']['location']['lat']) for result in data2['results'][:nb_result]],
    'name' : [result['name'] for result in data2['results'][:nb_result]],
    'formatted_address': [result['formatted_address'] for result in data2['results'][:nb_result]],
    'lat': [result['geometry']['location']['lat'] for result in data2['results'][:nb_result]],
    'lng' : [result['geometry']['location']['lng'] for result in data2['results'][:nb_result]],
    'photo_reference' : [result['photos'][0]['photo_reference'] for result in data2['results'][:nb_result]],
    'rating': [result['rating'] for result in data2['results'][:nb_result]],
    'user_ratings_total' : [result['user_ratings_total'] for result in data2['results'][:nb_result]],
}

#creation dataframe de la query:

df = pd.DataFrame(dict_adresses)
#connection à postgrey SQL à modifier avec create_engine(url)
engine = sql_user.engine

#Convertir la colonne "name" en liste
id_list = df["restaurant_id"].tolist()


#Générer les paramètres dynamiquement (:name0, :name1, etc.)
param_dict = {f"restaurant_id{i}": id for i, id in enumerate(id_list)}
placeholders = ", ".join(f":restaurant_id{i}" for i in range(len(id_list)))

#Requête SQL avec IN
query = text(f"SELECT * FROM restaurants WHERE restaurant_id IN ({placeholders})")

#Exécution de la requête
with engine.connect() as connection:
    result = connection.execute(query, param_dict)
    restaurants = result.fetchall()

restaurants = pd.DataFrame(restaurants)
display(restaurants)
if len(restaurants) !=0 :
    restaurants = set(restaurants["restaurant_id"].unique())  # Convertir en ensemble d'IDs
df_push = df.loc[~df["restaurant_id"].isin(restaurants)]
# Filtrer correctement

df_push.to_sql(name='restaurants', con=engine, if_exists = 'append', index=False)

,restaurant_id,name,lat,lng,formatted_address,photo_reference,rating,user_ratings_total,types
0,FLUNCH TOULOUSE GRAMONT43.6316772,FLUNCH TOULOUSE GRAMONT,43.6316772,1.4859998,"2 Chem. de Gabardie C.Cial Auchan, 31200 Toulo...",AVzFdbkiEuyyJS8hzWmsR4G1eNmnvqI0TkOCJ60M1vvCKC...,3.30,2483,None
1,Le Carré Gourmand43.6308066,Le Carré Gourmand,43.6308066,1.4843831,"2 Chem. de Gabardie, 31200 Toulouse, France",AVzFdbkyyGFN0yOoR05cKMMS3HVeTNsYC7dYuF9nb4nMAI...,4.10,921,None
2,Bowling Gramont43.6320117,Bowling Gramont,43.6320117,1.4864597,"Centre Commercial Auchan, 2 Chem. de Gabardie,...",AVzFdbl3Wj-lVjOLWqoavyvhA8o3TSeL1g0erP5Hdtqvvs...,4.10,2839,None
3,La Bièrothèque Gramont43.6344617,La Bièrothèque Gramont,43.6344617,1.4822392,"90 Chem. de Gabardie, 31200 Toulouse, France",AVzFdbkcd8f8mDbOmRqb81AY1t29ZcjVJUIcdeAiY63K8V...,4.30,1276,None
4,Crêp'eat Toulouse Gramont43.6312578,Crêp'eat Toulouse Gramont,43.6312578,1.4837159,"2 Chem. de Gabardie, 31200 Toulouse, France",AVzFdbk195Rs25rYMHNzXCmcdwx0-VTrwcMBqug8lZnyeb...,4.00,685,None
5,Chez Yvonne43.6268539,Chez Yvonne,43.6268539,1.4846236,"6 Av. Prat Gimont, 31130 Balma, France",AVzFdbmDvkxu0Uul5BdwFaUO6j6Lx4ZO26X8oepnwwRu-I...,4.50,2557,None
6,Le Dynastie Balma Gramont43.6317713,Le Dynastie Balma Gramont,43.6317713,1.4863686,"Centre Commercial Gramont, 2 Chem. de Gabardie...",AVzFdbnrqjkXZcSYDuahaVYoTih1YH_dQfrNcBZgnqhI0f...,4.00,1822,None
7,Mangos BALMA GRAMONT43.631436,Mangos BALMA GRAMONT,43.6314360,1.4826395,"2 Chem. de Gabardie, 31200 Toulouse, France",AVzFdbnySfarQnRQRFMfMHszfDX-J7MaukH2kR5CTSshTW...,4.50,208,None
8,BCHEF - TOULOUSE GRAMONT43.631473,BCHEF - TOULOUSE GRAMONT,43.6314730,1.4863957,"2 Chem. de Gabardie, 31200 Toulouse, France",AVzFdblvXFV08QcJd8sOHyxSDVSmE8zyJx4J46cmOAapZw...,3.00,293,None
9,Amafolia Restaurant Balma - Brasserie Méditerr...,Amafolia Restaurant Balma - Brasserie Méditerr...,43.6250931,1.4861163,"74 Rue Saint-Jean, 31130 Balma, France",AVzFdbkCfXXucFV4rWY6b_tcEaI9T1YmJN5w-nsQXkETyw...,4.50,3280,None


2

verification et identification client

In [67]:
from datetime import datetime

now = datetime.now()


def get_formatted_datetime():
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

# Appliquer la fonction et créer la colonne 'date'
df['date'] = df.apply(lambda row: get_formatted_datetime(), axis=1)

df.to_sql(name='query', con=engine, if_exists = 'append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "user_id" of relation "query" violates not-null constraint
DETAIL:  Failing row contains (1, null, FLUNCH TOULOUSE GRAMONT43.6316772, 2025-02-04 15:31:14+00, FLUNCH TOULOUSE GRAMONT, 43.6316772, 1.4859998, 2 Chem. de Gabardie C.Cial Auchan, 31200 Toulouse, France, AVzFdblB_S8-r-SIXRfXsGk_0g3P9D4Vh0TXiWaEM0hfg4WARZz6D6cKTX413_LA..., 3.30, 2484, null).

[SQL: INSERT INTO query (restaurant_id, name, formatted_address, lat, lng, photo_reference, rating, user_ratings_total, date) VALUES (%(restaurant_id)s, %(name)s, %(formatted_address)s, %(lat)s, %(lng)s, %(photo_reference)s, %(rating)s, %(user_ratings_total)s, %(date)s)]
[parameters: ({'restaurant_id': 'FLUNCH TOULOUSE GRAMONT43.6316772', 'name': 'FLUNCH TOULOUSE GRAMONT', 'formatted_address': '2 Chem. de Gabardie C.Cial Auchan, 31200 Toulouse, France', 'lat': 43.6316772, 'lng': 1.4859998, 'photo_reference': 'AVzFdblB_S8-r-SIXRfXsGk_0g3P9D4Vh0TXiWaEM0hfg4WARZz6D6cKTX4

In [65]:
df

,restaurant_id,name,formatted_address,lat,lng,photo_reference,rating,user_ratings_total,date
0,FLUNCH TOULOUSE GRAMONT43.6316772,FLUNCH TOULOUSE GRAMONT,"2 Chem. de Gabardie C.Cial Auchan, 31200 Toulo...",43.631677,1.486000,AVzFdblB_S8-r-SIXRfXsGk_0g3P9D4Vh0TXiWaEM0hfg4...,3.3,2484,2025-02-04 15:23:56
1,Le Carré Gourmand43.6308066,Le Carré Gourmand,"2 Chem. de Gabardie, 31200 Toulouse, France",43.630807,1.484383,AVzFdbkM4FeCoPbotDnpCIDU5mGQUp9Tnmt_b6ztAC72Ws...,4.1,921,2025-02-04 15:23:56
2,Bowling Gramont43.6320117,Bowling Gramont,"Centre Commercial Auchan, 2 Chem. de Gabardie,...",43.632012,1.486460,AVzFdbna-KORTYT0_PRawxGY8uk9oB43jnAPe8m1el0Xg_...,4.1,2839,2025-02-04 15:23:56
3,Jerry's - Restaurant & Brasserie43.6321848,Jerry's - Restaurant & Brasserie,"2 Chem. de Gabardie, 31200 Toulouse, France",43.632185,1.486384,AVzFdblR9o9bq80DS_biaCXvkLpul50Hfj3drlzXPKYeTc...,4.7,628,2025-02-04 15:23:56
4,Crêp'eat Toulouse Gramont43.6312578,Crêp'eat Toulouse Gramont,"2 Chem. de Gabardie, 31200 Toulouse, France",43.631258,1.483716,AVzFdbnnKjHkW6unTXx67iZnS7z-vR0wyoZLdzAlSnMkil...,4.0,685,2025-02-04 15:23:56
5,La Cafétéria43.6012356,La Cafétéria,"37 Rue Boulbonne, 31000 Toulouse, France",43.601236,1.447771,AVzFdbnxbu2JrLK0IWIILK_mvmVMS-OdVCpryP0C9Clavc...,4.3,268,2025-02-04 15:23:56
6,La Bièrothèque Gramont43.6344617,La Bièrothèque Gramont,"90 Chem. de Gabardie, 31200 Toulouse, France",43.634462,1.482239,AVzFdbkg8kGeSQcjml3SXCgxTC_Z4r8JAfBdUqRHMya5fg...,4.3,1276,2025-02-04 15:23:56
7,Chez Yvonne43.6268539,Chez Yvonne,"6 Av. Prat Gimont, 31130 Balma, France",43.626854,1.484624,AVzFdbnF4i-b9G6FquAIlxTJ1B2mHK2lrmfxlvEB54ago2...,4.5,2557,2025-02-04 15:23:56
8,KFC Toulouse Gramont (Balma)43.62772160000001,KFC Toulouse Gramont (Balma),"81 Rue Saint-Jean, 31130 Balma, France",43.627722,1.482654,AVzFdbmLf5wIOHc1cZuyy2WSka_cwfxpGVhSluBN6TXJhq...,3.7,4417,2025-02-04 15:23:56
9,Le Dynastie Balma Gramont43.6317713,Le Dynastie Balma Gramont,"Centre Commercial Gramont, 2 Chem. de Gabardie...",43.631771,1.486369,AVzFdbkqFgildlCkgwJNk7nXiuY9neRFd4ketvwRU9otKT...,4.0,1822,2025-02-04 15:23:56


In [103]:
#verification idenfication


identification = 0

while identification == 0:

    email = input('e-mail')
    secret_pass = input('Password')
    query = text("SELECT * FROM user WHERE mail = :email AND password = :secret_pass")
        #Exécution de la requête
    with engine.connect() as connection:
        result = connection.execute(query, {'email': email, 'secret_pass': secret_pass})
        identification = result.fetchall()
    identification = pd.DataFrame(identification)
    display(identification)
    if len(identification) == 0 :
        display('Mail ou mot de passe incorrect')
        identification = 0
    else :
        identification = len(identification)

display('affichage page principale')

    

""


'Mail ou mot de passe incorrect'

,id,nom,prenom,mail,password
0,1,Martin,Jean,jean.martin@test.com,@test.com


'affichage page principale'

In [3]:
test = sql_user.is_authentificate('noah.brown@test.com','@Test.com31')

In [5]:
test[1]

3

In [139]:
import re

new_user_mail = input('e-mail')
new_user_name = input('Name')
new_user_lastname = input('Lastname')
new_user_secret_pass = input('Password')

def checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass):
    #regex de qui vont servir de verification
    regex_mail = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    regex_password = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'
    query = text("SELECT * FROM user WHERE mail = :new_user_mail")
    with engine.connect() as connection:
            result = connection.execute(query, {'new_user_mail': new_user_mail})
            creation = result.fetchall()
    creation = pd.DataFrame(creation)
    if len(creation) == 0:
        creation.to_sql(name='user', con=engine, if_exists = 'append', index=False)
    else :
        return "email déjà connu merci de vous identifier"  
    if re.match(regex_mail, new_user_mail):
        new_user_mail = True
        if len(new_user_name) !=0:
            new_user_name = True
            if len(new_user_lastname) !=0:
                new_user_lastname = True
                if re.match(regex_password, new_user_secret_pass):
                    new_user_secret_pass = True
                else:
                    new_user_secret_pass = False
            else:
                new_user_lastname = False
        else:
            new_user_name = False  
    else:
        new_user_mail = False
    chek_in = [new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass]
    return chek_in

checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass)



[True, True, True, True]

In [11]:
import re

# Exemple de mot de passe à tester
password = '"rèsT"658!'

# Expression régulière pour valider un mot de passe
regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

# Vérification du mot de passe
if re.match(regex, password):
    print("Mot de passe valide")
else:
    print("Mot de passe invalide")


Mot de passe valide


In [26]:
class Checker:
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    password_regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

    def __init__(self, engine):
        self.engine = engine

    def is_email_known(self, email):
        query = text("SELECT COUNT(*) FROM user WHERE email = :email")
        with self.engine.connect() as connection:
            result = connection.execute(query, {'email': email})
            count = result.scalar()
        return count > 0

    @staticmethod
    def is_valid_email(email):
        return re.match(Checker.email_regex, email) is not None

    @staticmethod
    def is_valid_password(password):
        return re.match(Checker.password_regex, password) is not None

    @staticmethod
    def is_valid_name(name):
        return bool(name) and all(char.isalpha() or char.isspace() for char in name)

    @staticmethod
    def is_valid_lastname(lastname):
        return bool(lastname) and all(char.isalpha() or char.isspace() for char in lastname)

    def validate_user(self, email, password, name, lastname):
        if not self.is_valid_email(email):
            print("Mail invalide")
            return False
        if not self.is_valid_password(password):
            print("Mot de passe invalide merci de renseigner minimum 8 caractères dont un chiffre, majuscule et symbole")
            return False
        if not self.is_valid_name(name):
            print("Name invalid")
            return 
        if not self.is_valid_lastname(lastname):
            print("Lastname invalid")
            return 
        return True
    # alternatif
    def val(self, email, password, name, lastname):
        return not self.is_valid_email(email) and not self.is_valid_password(password) and not self.is_valid_name(name) and not self.is_valid_lastname(lastname)

    @staticmethod
    def add_user(email, password, name, lastname):
        dict_user = {'nom': [name], 'prenom': [lastname], 'mail': [email], 'password': [password]}
        df_user = pd.DataFrame(dict_user)
        df_user.to_sql(name='user', con=engine, if_exists = 'append', index=False)
    
    
    def is_authentificate(self, email, password):
        query = text("SELECT COUNT(*) FROM user WHERE mail = :email and password = :password")
        with self.engine.connect() as connection:
            result = connection.execute(query, {'email': email, 'password': password})
            count = result.scalar()
        return count > 0

# Exemple d'utilisation
email = "r.meson@test.com"
password = "WCS_Rémi_2025"
name = "Jean"
lastname = "Dupont"



In [2]:
# Exemple d'utilisation
email = "r.meson@test.com"
password = "WCS_Rémi_2025"
name = "Jean"
lastname = "Dupont"



In [3]:
from checker import Checker

checker= Checker()

checker.is_email_known(email)

False

In [35]:


if checker.is_email_known(email):
    print("Mail déjà connu merci de vous identifier")
else:
    if checker.validate_user(email, password, name, lastname):
        checker.add_user(email, password, name, lastname)

Mail déjà connu merci de vous identifier


In [3]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------------------------------ --- 2.4/2.6 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
